Cleaning crime data such that:
*   Get latitude and longitude coordinates for each data point
*   Convert string representation of date and time to numerical
*   Group offenses into categories


In [49]:
# Installations for geopandas
!apt update
!apt upgrade
!apt install gdal-bin python-gdal python3-gdal 
# Install rtree - Geopandas requirment
!apt install python3-rtree 
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git
# Install descartes - Geopandas requirment
!pip install descartes 

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (106 kB/s)
Reading package lists... Done
Building dependency tree       
Reading s

In [0]:
# Download data and relevant packages
import os
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import timedelta
import geopandas as gpd

csv_url = 'https://opendata.arcgis.com/datasets/d1877e350fad45d192d233d2b2600156_6.csv'
crime_df = pd.read_csv(csv_url)
crime_df = crime_df.drop(['HourReported', 'Agency'], axis = 1)

In [51]:
# Convert string representations of data and time to numerical

dt_string = "2019-08-29T17:47:31.000Z"
# Considering date is in dd/mm/yyyy format
dt_object1 = datetime.strptime(dt_string, "%Y-%m-%dT%H:%M:%S.000Z")
print("dt_object1 =", dt_object1)

# Create a column with the date reported as datetime objects
crime_df['TimeReported'] = crime_df['DateReported'].apply(lambda dt_string: datetime.strptime(dt_string, "%Y-%m-%dT%H:%M:%S.000Z"))
crime_df = crime_df.drop('DateReported', axis = 1)

crime_df.head()

dt_object1 = 2019-08-29 17:47:31


,RecordID,Offense,IncidentID,BlockNumber,StreetName,TimeReported
0,1001,Larceny - From Motor Vehicle,201900024682,400.0,5TH ST SW,2019-08-29 17:47:31
1,1002,Assault Intimidation,201900024676,1900.0,EMMET ST N,2019-08-29 16:51:37
2,1003,Larceny - All Other,201900024664,1300.0,CARLTON AVE,2019-08-29 15:45:04
3,1004,Lost/FoundProperty,201900024659,200.0,RIDGE MCINTIRE RD,2019-08-29 15:03:52
4,1005,Assault Simple,201900024652,600.0,CABELL AVE,2019-08-29 14:08:09


In [59]:
# Given a list of possible types of offenses, returns dataframe only containing rows with the offense
# Case insensitive

def select_crime(df, crime_name):
  selected_df = df[df['Offense'].str.contains(crime_name, case=False)] 
  print(selected_df.head())

    RecordID                       Offense  ...      StreetName        TimeReported
0       1001  Larceny - From Motor Vehicle  ...       5TH ST SW 2019-08-29 17:47:31
2       1003           Larceny - All Other  ...     CARLTON AVE 2019-08-29 15:45:04
8       1009           Larceny - All Other  ...   MONTICELLO RD 2019-08-28 21:19:03
11      1012  Larceny - From Motor Vehicle  ...  MONTICELLO AVE 2019-08-28 04:22:28
17      1018           Larceny - All Other  ...     WERTLAND ST 2019-08-27 21:39:21

[5 rows x 6 columns]


In [37]:
# Get latitude and longitude coordinates of each data point
# single address
locator = Nominatim(user_agent=”myGeocoder”)
location = locator.geocode(“Champ de Mars, Paris, France”)
print(“Latitude = {}, Longitude = {}”.format(location.latitude, location.longitude))

# multiple addresses
df = pd.read_csv(“addresses.csv”)
df.head()
# Karlaplan 13,115 20,STOCKHOLM,Stockholms län, Sweden
# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
# 2- - create location column
df['location'] = df['ADDRESS'].apply(geocode)
# 3 - create longitude, laatitude and altitude from location column (returns tuple)
df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)
# 4 - split point column into latitude, longitude and altitude columns
df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)
#cleaned table with latitude and longitude
df = df.drop([‘Address1’, ‘Address3’, ‘Address4’, ‘Address5’,’Telefon’, ‘ADDRESS’, ‘location’, ‘point’], axis=1)
df.head()

# visualization
map1 = folium.Map(
    location=[59.338315,18.089960],
    tiles='cartodbpositron',
    zoom_start=12,
)
df.apply(lambda row:folium.CircleMarker(location=[row["latitude"], row["longitude"]]).add_to(map1), axis=1)
map1


SyntaxError: ignored